# Product $\sigma$-Algebra

This tutorial is part of a bigger series on probabilistic machine learning and aids in understanding product $\sigma$-algebras.
At the end of this notebook, you will understand...

- The Definition of the $\sigma$-Algebra
- Why these concepts matter, especially for robotics
- How to apply them using the package   


## Motivation

Since studying concepts like the $\sigma$-Algebra is dry, I want to motivate you on why this matters and you should take the time to think about the $\sigma$-Algebra and specially the Product $\sigma$-Algebra:

- **Foundations of probability theory**: $\sigma$-algebras are the building blocks for defining probability in a rigorous way. By understanding them, you gain a deeper understanding of how probabilities are assigned to events.
- **Working with complex events**:  In real-world scenarios, events can be intricate. Sigma algebras allow you to describe not just simple events but also unions, intersections, and complements of these events, giving you a powerful tool to analyze probabilities of more complex situations.
- **Connection to advanced math**: Sigma algebras bridge the gap between set theory and advanced mathematical concepts like measure theory and integration. Studying them opens doors to these powerful tools used in various scientific fields.


If you are interested in robotics, it is also important since:

- **Reasoning with uncertainty**: Robots often operate in environments with uncertainty. $\sigma$ algebras provide a mathematical foundation to represent uncertain events and reason about the probability of different events happening (like sensor readings or obstacles appearing).
- **Decision making under probability**:  Many robotic tasks involve making decisions based on probabilities. By understanding $\sigma$-algebras, you can build algorithms that consider the chance of different outcomes and choose the action with the highest probability of success.
- **Planning and control under uncertainty**:  Planning robot actions often requires considering various possibilities. $\sigma$-algebras allow you to create probabilistic models of the environment, enabling robots to plan and control their movements while accounting for uncertainties.

Research has shown that events that are described by independent constraints (rules) are most likely the only events where probability estimation is tractable.
Spaces that are constructed by independent constraints are called product spaces.
Understanding the shape of such events is a key competence to building (new) tractable probabilistic models.
In this tutorial, we will work with some visualizations of said events, to get a better understanding of their shape, behavior and math.

In [45]:
from random_events.variables import Symbolic, Continuous, Integer
from random_events.events import Event, ComplexEvent
import portion
import plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objects as go
from itertools import chain, combinations, product

We begin by reciting the definition of a $\sigma$-algebra.

## $\sigma$-Algebra

A $\sigma$ algebra is a set of sets that contains all set-differences that can be constructed by combining arbitrary subsets of said set. Furthermore, it contains all countable unions of sets and all infinite intersection of the set. More formally, according to Kolmogoroff: 

Let $E$ be a space of elementary events. Consider the powerset $2^E$ and let $\Im \subset 2^E$ be a set of subsets of $E$. Elements of $\Im$ are called random events. If $\Im$ satisfies the following properties,it is called a $\sigma$-algebra.

1. $E \in \Im$
2. $(A, B) \in \Im \Rightarrow (A - B) \in \Im$
3. $(A_1, A_2, ... \in \Im) \Rightarrow \left( \bigcup_{i=1}^\mathbb{N} A_i \in \Im \wedge \bigcap_{i=1}^\infty A_i \in \Im \right)$

The tuple $(E, \Im)$ is called a measurable space.

An example of such a set of sets is the following:

In [46]:
def powerset(iterable):
    s = list(iterable)
    result = list(chain.from_iterable(combinations(s, r) for r in range(len(s) + 1)))
    return [set(x) for x in result]


E = {"a", "b", "c"}
powerset_of_E = powerset(E)
powerset_of_E

We can see that this is a correct $\sigma$-algebra by verifying all axioms. First, check if it contains the space of elementary Events $E$:

In [47]:
E in powerset_of_E

Next, check if it contains all set differences:

In [48]:
for A, B in combinations(powerset_of_E, 2):
    if A - B not in powerset_of_E:
        print(f"Set difference {A - B} not in powerset")

Finally, check if it contains all countable unions and intersections:

In [49]:
for A, B in combinations(powerset_of_E, 2):
    if A.union(B) not in powerset_of_E:
        print(f"Union {A.union(B)} not in powerset")
    if A.intersection(B) not in powerset_of_E:
        print(f"Intersection {A.intersection(B)} not in powerset")

We have constructed a $\sigma$-algebra. This is a very simple example, but it is important to understand the concept of such a system of sets.
As you can probably imagine, it is very inefficient to work with powersets of sets due to their exponential size. That's why I introduce the concept of product $\sigma$-algebras.

Product $\sigma$-algebras are constructed by taking the cartesian product of sets and then constructing the $\sigma$-algebra on the resulting set.
In this package, we generate product algebras from a viewpoint of classical machine learning. In machine learning scenarios we typically have a set of variables that we want to reason about. Random Events also start there. Let's start by defining some variables.

In [50]:
item = Symbolic("item", ["bowl", "cup", "spoon"])
color = Symbolic("color", ["blue", "green", "red"])
print(item)
print(color)

The variables we just constructed consisted of a name and a set of possible values, the so-called domain.
While the name is just an identifier, the domain is the set of elementary events, as described in the definition of a $\sigma$-algebra.
Regarding the formal terms from above, we can write a variable as a measurable space $(\text{variable.domain}, 2^\text{variable.domain})$, which means in common words that everything inside the domain is a possible and every combination of the things inside the domain is possible.

Forming combinations of those two variables introduces the product algebra.

## Product $\sigma$-Algebra

Let $(E_1,\Im_1)$ and $(E_2,\Im_2)$ be measurable spaces.
The product $\sigma$-algebra of $\Im_1$ and $\Im_2$ is denoted $\Im_1 \otimes \Im_2$, and defined as:
$\Im_1 \otimes \Im_2 := \sigma(\{S_1 \times S_2 : S_1 \in \Im_1 \wedge S_2 \in \Im_2\})$
where $\sigma$ denotes generated $\sigma$-algebra and $\times$ denotes Cartesian product.
This is a $\sigma$-algebra on the Cartesian product $E_1 \times E_2$.

An example of this product algebra is the combination of the item and color variables.

In [51]:
product_E = product(item.domain, color.domain)
list(product_E)

However, as these are already 9 elementary events the powerset contains $2^9 = 512$ elements. This is not feasible to work with. Hence, a better description of a subset of the powerset is needed.
This is where the concept of events comes into play. Events are subsets of the powerset that are constructed by constraints on the variables. The event they describe is given by the Cartesian product of all elements within the constraints.

In [52]:
event = Event({item: ("bowl", "cup"), color: "blue"})
list(product(*event.values()))

Unfortunately, a union of such event cannot be accurately described by a single event. Consider the following 

In [53]:
event1 = Event({item: "bowl", color: "blue"})
event2 = Event({item: "cup", color: "red"})

If the union of these events is constructed for every variable, one would obtain the following event

In [54]:
event_union = Event({item: ("bowl", "cup"), color: ("blue", "red")})
event_union

However, this is not the union of the two events. This union contains the event ("blue", "cup"), which was not part of any of the above events. Hence the real union is constructed this way:  

In [55]:
real_event_union = event1 | event2
str(real_event_union)

The correct union is a complex event. A complex event is a union of disjoint events.

## Connections to Logic

Algebraic concepts are hard to grasp. Since you, the reader, are very likely a Computer Scientists I will re-explain a random event from the perspective of logic.
We can rewrite the assignment of a variable to a set as a boolean variable. For example,
$Item_{\{\text{bowl}, \text{cup}\}} = item \in \{\text{bowl}, \text{cup}\}$
is a boolean variable that is true if the item is a bowl or a cup.
We can rewrite the statement of the union as logical statement.
$$ \left( Item_{\{\text{bowl}\}} \land Color_{\{\text{blue}\}} \right) \lor \left( Item_{\{\text{cup}\}} \land Color_{\{\text{red}\}} \right) $$
This logical statement describes either a blue bowl or a red cup.
The complex random events can always be thought of as a disjunction of conjunctions, hence a logical statement in the [disjunctive normal form](https://en.wikipedia.org/wiki/Disjunctive_normal_form).

## Continuous Domains

Getting a better understanding for such abstract concepts is best done through visualisations. Hence, we will now work with continuous variables.
In continuous variables, the possible values are intervals. This package uses portion to represent intervals. Let's get some hands on by defining continuous variables.

In [56]:
x = Continuous("x")
y = Continuous("y")

rectangle_event = Event({x: portion.closed(2, 3), y: portion.closed(10, 15)})
fig = go.Figure(rectangle_event.plot(), rectangle_event.plotly_layout())
fig.update_layout(title= "Rectangle event in 2D")
fig.show()

We can see that the described event is a rectangle in the x-y-plane. In fact, for higher dimensions, the described event will always be a hyper-rectangle.
Shapes like triangles, circles, etc. are not possible since they are made from dependent constraints. A circle with radius r, for example, can be described by the constraint `x^2 + y^2 <= r^2`, which is not independent.

The only upper class of more complicated shapes can be constructed by defining more complex, independent constraints.

In [57]:
complex_event = Event({x: portion.closed(2, 3) | portion.closed(4, 5) | portion.closed(6,7), y: portion.closed(10, 15) | portion.closed(25, 27)})
complex_event

Let's have a graphical look at it.

In [58]:
fig = go.Figure(complex_event.plot(), complex_event.plotly_layout())
fig.update_layout(title= "Complex event in 2D")
fig.show()

The generalization of such events in higher dimensions results in hyper-rectangles. Let's visualize the transition from 2D to 3D to get a feel for how these shapes behave.

In [59]:
# extend previous event by 3rd dimension
z = Continuous("z")
complex_event_3d = complex_event.copy()
complex_event_3d[z] = portion.closed(1, 3) | portion.closed(4, 4.5) | portion.closed(10,11.5)
fig = go.Figure(complex_event_3d.plot(), complex_event_3d.plotly_layout())
fig.update_layout(title= "Complex event in 3D")
fig.show()

Unfortunately, the visualization of more than three dimensions is infeasible. The behavior of such events in higher dimensions yet remains the same. New constraints just add another dimension to the
rectangle. The patterns that can be created by having multiple intervals also generalize the same way as it did from 2D to 3D.

The final component to look at, is the outer space. When the complement of a rectangular event is created, the result is a set of rectangles that are not part of the original event.
This may look like this.

In [60]:
event = Event({x: portion.open(0, 1), y: portion.open(0, 1)})
complement = event.complement()
limiting_event = Event({x: portion.closed(-1, 2), y: portion.closed(-1, 2)})
result = complement.intersection(ComplexEvent([limiting_event]))
fig = go.Figure(result.plot(), result.plotly_layout())
fig.show()

In 3D, the outer event looks weird, but it is just the complement of the original event. You can use the interactive zoom functionality to see the missing inner event.

In [61]:
event = Event({x: portion.closed(0, 1),
               y: portion.closed(0, 1),
               z: portion.closed(0, 1)})
complement = event.complement()
limiting_event = Event({x: portion.closed(-1, 2),
                        y: portion.closed(-1, 2),
                        z: portion.closed(-1, 2)})
result = complement.intersection(ComplexEvent([limiting_event]))
fig = go.Figure(result.plot(), result.plotly_layout())
fig.show()

In case you didn't find the inner, missing event, here is the outer event cut open. 

In [62]:
cut_result = result.intersection(Event({y: portion.closed(-1, 1)}))
fig = go.Figure(cut_result.plot(), cut_result.plotly_layout())
fig.show()

## Complement of the Product Algebra

[This](https://www.math.ucdavis.edu/~hunter/m206/ch4_measure_notes.pdf) mentions that the complement of an element of the product measure is constructed by
$$
    (A \times B)^c = (A^c \times B) \cup (A \times B^c) \cup (A^c \times B^c).
$$
It is easy to see that this construction would produce exponential many elements with respect to the number of variables. This is unfortunate.
However, the correct complement can be formed with linear many terms, which is way more efficient. The following equations describe a proof by induction on how that can be done.

Let
\begin{align*}
    \mathbb{A} &= A \cup A^c \, , \\
    \mathbb{B} &= B \cup B^c \text{ and }\\
    \mathbb{C} &= C \cup C^c.
\end{align*}

### Induction Assumption

\begin{align*}
    (A \times B)^c = (A^c \times \mathbb{B}) \cup (A \times B^C)
\end{align*}
Proof:
\begin{align*}
    (A \times B)^c &= (A^c \times B) \cup (A \times B^c) \cup (A^c \times B^c) \\
    &= (A^c \times B) \cup (A^c \times B^c) \cup (A \times B^c) \\
    &= ( A^c \times (B \cup B^c) ) \cup   (A \times B^c) \\
    &= (A^c \times \mathbb{B}) \cup (A \times B^C) \square
\end{align*}

### Induction Step

\begin{align*}
    (A \times B \times C)^c = (A^c \times \mathbb{B} \times \mathbb{C}) \cup (A \times B^C \times \mathbb{C} ) \cup (A \times B \times C^c)
\end{align*}
Proof:
\begin{align*}
    (A \times B \times C)^c &= (A^c \times B \times C) \cup (A \times B^c \times C) \cup (A \times B \times C^c) \cup 
    (A^c \times B^c \times C) \cup (A^c \times B \times C^c) \cup (A \times B^c \times C^c) \cup 
    (A^c \times B^c \times C^c) \\
    &= (C \times \underbrace{(A^c \times B) \cup (A \times B^c) \cup (A^c \times B^c))}_{\text{Induction Assumption}} \cup
    (C^c \times  \underbrace{(A^c \times B) \cup (A \times B^c) \cup (A^c \times B^c))}_{\text{Induction Assumption}} \cup (A \times B \times C^c) \\
    &= (C \times (A^c \times \mathbb{B}) \cup (A \times B^C)) \cup 
       (C^c \times (A^c \times \mathbb{B}) \cup (A \times B^C)) \cup (A \times B \times C^c)\\
    &= 
\end{align*}


## Application of the Product Algebra

You may ask yourself where the product algebra matters in real applications.
Consider your kitchen. You most likely have some regions where you are able to stand, and some regions where you can't.
If you look at floor plan of your kitchen, you could perhaps describe it as the following event.  

In [63]:
kitchen = Event({x: portion.closed(0, 6.6), y: portion.closed(0, 7)})
refrigerator = Event({x: portion.closed(5, 6), y: portion.closed(6.3, 7)})
top_kitchen_island = Event({x: portion.closed(0, 5), y: portion.closed(6.5, 7)})
left_cabinets = Event({x: portion.closed(0, 0.5), y: portion.closed(0, 6.5)})

center_island = Event({x: portion.closed(2, 4), y: portion.closed(3, 5)})

occupied_spaces = refrigerator | top_kitchen_island | left_cabinets | center_island
fig = go.Figure(occupied_spaces.plot(), occupied_spaces.plotly_layout())
fig.show()

Now posing the question on where you can stand in your kitchen, you can simply calculate the complement of the occupied space with the kitchen.

In [64]:
free_space = kitchen.difference(occupied_spaces)
fig = go.Figure(free_space.plot(), free_space.plotly_layout())
fig.show()

Now this already sounds somewhat useful. However, just the events are of limited use. The real power of the product algebra comes when you start to calculate probabilities of events.
For this, you can check out this tutorial on [probability theory](https://probabilistic-model.readthedocs.io/en/latest/examples/probability_theory.html).